In [143]:
import unicodedata

def remove_accents(row):
    nfkd_form = unicodedata.normalize('NFKD', row).lower()
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [144]:
import gzip, pickle

with gzip.open('dicionario_wb.dict.gz','rb') as fp:
    dicionario = pickle.load(fp)
    fp.close()
    
with gzip.open('lista_nomes_conjugado.dict.gz','rb') as fp:
    lista_nomes_conjugado = pickle.load(fp)
    fp.close()

In [145]:
def porcentagem(parte,total):
        #parte = contador_nomes_retirados
        #total = contador_nomes_HAREM
        porcentagem = None
        if total != 0:  
            porcentagem = (parte / total)*100
        
        print("Porcentagem: ", porcentagem)       

In [146]:
def porcentagem_parc(parte,total):
    porcentagem_parc = None
    if total != 0:
        porcentagem_parc = (parte/total)*100
        
    print("Porcentagem Parcial: ",porcentagem_parc)    

In [217]:
import re
import xml.etree.ElementTree as ET

def remove_names(row, filename, debug=False):
    a = os.getcwd()+'/HAREM_EVO/HAREMRELREL.xml'
    #f = open(a)
    #golden_lines = f.readlines()
    tree = ET.parse(a)
    palavras_removidas = []
    root = tree.getroot()
    #for child in root:
    #    print(child.get('DOCID'))
    
    ## Contadores
    contador_nomes_HAREM = 0
    contador_nomes_retirados = 0
    contador_nomes_parcialmente_retirados = 0    
    
    
    ## Separa cada uma das palavras
    sentencas = row.split('\n')
    #print(sentencas)
    j = 0
    for s in sentencas:
        palavras = s.split(' ')
        ## Percorre todas as palavras
        for i, p in enumerate(palavras):           

            ## Remove acentos
            ## Coloca palavras em minúsculas 
            ## Substitue caracteres especiais por espaço
            p = remove_accents(p).lower()
            n = re.sub(r'\W',' ',p).strip()

            #for palavra in dicionario:

                # verifica se no inicio ou no fim há um nome
                # remove do dicionario a palavra toda se houver 
                #if parte.rfind(nome) == (len(parte) - len(nome)): *** TEM QUE VERIFICAR A PALAVRAAAAAA
                    #print ('NomeDic:', nome )
                    #remove(parte.rfind(nome))    QUERO QUE REMOVA A PALAVRA DO DICIONARIO, NAO O NOME DELA!
        


            ## Substitue Nomes
            if n in lista_nomes_conjugado:
                #print ('nomes > ',n)
                if debug: print("---Nome Removido---")
                if debug: print(n, p)
                palavras[i] = p.lower().replace(n,'NoInfo')
                palavras_removidas.append(n)
            
            else:
                for parte in n.split(' '):

                    #print(parte)
                    ### Substitue nomes no meio de palavras (que não existem no dicionário)
                    if not remove_accents(parte) in dicionario:
                        #if len(parte) > 3:
                            #print('No Dict: ',parte)
                        for nome in lista_nomes_conjugado:
                            in_list = False
                            if parte == nome: #NÃO PEGA O NOME INTEIRO CORRETAMENTE D:
                                p = p.replace(nome,'NoInfo')
                                #print('Inteiro', parte,'*', nome)
                                in_list = True
                                palavras_removidas.append(nome)
                        for nome in lista_nomes_conjugado:
                            ## Remove nomes no meio da palavra
                            #if len(nome) > 3 and parte.find(nome) >= 0:
                            #    print('find:',parte,'|',nome)
                            ## Remove nomes no início da palavra
                            if len(nome) > 3 and parte.find(nome) == 0: 
                                #print('Inicio:',parte,'|',nome)
                                if not in_list:
                                    in_list = True
                                    palavras_removidas.append(nome)
                                p = p.replace(nome,'NoInfo ')
                            ## Remove nomes no final da palavra
                            if len(nome) > 3 and parte.rfind(nome) > 0 and parte.rfind(nome) == (len(parte)-len(nome)):
                                #print('Final:', parte,'-',nome)
                                if not in_list:
                                    in_list = True
                                    palavras_removidas.append(nome)
                                p = p.replace(nome,' NoInfo ')

                        palavras[i] = p.replace(nome,' NoInfo ')


            ## Substitue os números de registros
            num = re.findall(r'\d*',p)[0]
            if len(num) > 7:
                #print (p)
                palavras[i] = 'NoInfo'

            ## Substitue o Leito
            if len(num) >= 3 and palavras[i-1].lower() == 'leito:':
                #print(p, palavras[i-1])
                palavras[i] = 'NoInfo'
        s = ' '.join(palavras)
        
        prev_find = s.find('NoInfo')

        
    #while prev_find != -1:
    #Terminar de comparar os nomes retirados dos textos com os do HArem
    palavras_removidas = set(palavras_removidas)
    if debug: print('---- Nomes Removidos ----')
    if debug: print(palavras_removidas)
    for doc in root:
        #print("Filename", filename)
        if doc.get('DOCID') == filename:
            #print("DOCID", filename)
            for paragraph in doc:
                for em in paragraph:
                    if em.get("CATEG") == "PESSOA" and em.get('TIPO') == "INDIVIDUAL":
                        #remover acentos

                        em_temp = re.sub(r'\W',' ',em.text).strip()
                        
                        nomes = em_temp.split()
                        
                        # lista de nomes sem acento
                        for nome in nomes:

                            #p = remove_accents(p).lower()

                            nome_sem_acento = remove_accents(nome).lower()

                            contador_nomes_HAREM += len(nomes)

                            

                            #for nome in nomes:
                            if debug: print("-----Comparando Nomes----")
                            if debug: print(nome)
                            #print(palavras_removidas)
                            for palavra_removida in palavras_removidas:
                                if nome_sem_acento == palavra_removida.lower():
                                    contador_nomes_retirados += 1
                                    if debug: print(nome_sem_acento, " |t ", palavra_removida)
                                elif nome_sem_acento.find(palavra_removida.lower()) != -1:
                                    contador_nomes_parcialmente_retirados += 1
                                    if debug: print(nome_sem_acento, " |p ", palavra_removida)
            break
                                                    
            #for gl in golden_lines:
            #    if gl.find('DOCID='):
            #        doc_id = gl[gl.find('"')+1:gl.findr('"')]
            #        print(doc_id)
            #        if doc_id == filename:
                        
            #E aí comparar com cada palavras retirada em palavras_removidas para ver se está correta.
            prev_find = s.find('NoInfo', prev_find+1)
            j = j + 1
        #break
    
    
    print("Nomes no HAREM: ", contador_nomes_HAREM, " | Nomes Retirados: ", contador_nomes_retirados, " | Nomes Parcialmente Retirados: ", contador_nomes_parcialmente_retirados, " |Número de Erros: ", contador_nomes_HAREM - contador_nomes_parcialmente_retirados - contador_nomes_retirados)     
    
    porcentagem(contador_nomes_retirados, contador_nomes_HAREM)
    porcentagem_parc(contador_nomes_parcialmente_retirados, contador_nomes_HAREM)
    
    ## Une todas as palavras de volta
    #return ' '.join(palavras)
    return contador_nomes_HAREM, contador_nomes_retirados + contador_nomes_parcialmente_retirados

In [228]:
import os, pickle
import xml.etree.ElementTree as ET

a = os.getcwd()+'/HAREM_EVO/HAREMRELREL.xml'
#f = open(a)
#golden_lines = f.readlines()
tree = ET.parse(a)
palavras_removidas = []
root = tree.getroot()
    
c = os.listdir(os.getcwd()+'/HAREM_EVO/')
a = os.getcwd()+'/HAREM_EVO/'
conteudo = []
achou = False
total_i = 0
removidos_i = 0
ar_total = []
ar_removidos = []

for p in c:
    if p.endswith(".txt"):
        file = open(a+p)
        print(p)
        achou = False
        for doc in root:
            if doc.get('DOCID') == p.replace('.txt', ''):
                #print (doc)
                achou = True
                total_i, removidos_i = remove_names(file.read(), p.replace('.txt', ''), False)
                ar_total.append(total_i)
                ar_removidos.append(removidos_i)
                break
        
        if not achou: print('Não achou!')
        #print(file.read())"/textosnerp/"
        #conteudo.append(file.read())
        file.close()
    
    #if achou: break

#print(conteudo)

hub-57257.txt
Nomes no HAREM:  94  | Nomes Retirados:  23  | Nomes Parcialmente Retirados:  1  |Número de Erros:  70
Porcentagem:  24.46808510638298
Porcentagem Parcial:  1.0638297872340425
cha-81393.txt
Não achou!
cha-73181.txt
Não achou!
cha-46556.txt
Não achou!
cha-23475.txt
Não achou!
cha-49273.txt
Não achou!
cha-85455.txt
Não achou!
cha-17328.txt
Não achou!
cha-54730.txt
Não achou!
ric-58766.txt
Nomes no HAREM:  1  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:  0  |Número de Erros:  1
Porcentagem:  0.0
Porcentagem Parcial:  0.0
cha-33887.txt
Não achou!
cha-40076.txt
Não achou!
cha-99622.txt
Não achou!
wpt-1000772700099419796.txt
Nomes no HAREM:  72  | Nomes Retirados:  29  | Nomes Parcialmente Retirados:  0  |Número de Erros:  43
Porcentagem:  40.27777777777778
Porcentagem Parcial:  0.0
cha-26556.txt
Não achou!
evo1510.txt
Não achou!
evo1500.txt
Não achou!
evo877.txt
Não achou!
evo876.txt
Não achou!
evo875.txt
Não achou!
evo874.txt
Não achou!
evo872.txt
Não achou!
evo871.

Nomes no HAREM:  25  | Nomes Retirados:  3  | Nomes Parcialmente Retirados:  0  |Número de Erros:  22
Porcentagem:  12.0
Porcentagem Parcial:  0.0
cha-49789.txt
Não achou!
wpt-1005220936254463458.txt
Nomes no HAREM:  9  | Nomes Retirados:  2  | Nomes Parcialmente Retirados:  0  |Número de Erros:  7
Porcentagem:  22.22222222222222
Porcentagem Parcial:  0.0
cha-47217.txt
Não achou!
cha-87179.txt
Não achou!
cha-75283.txt
Não achou!
cha-59248.txt
Não achou!
cha-75128.txt
Não achou!
cha-25368.txt
Não achou!
cha-11350.txt
Não achou!
hub-39869.txt
Nomes no HAREM:  22  | Nomes Retirados:  2  | Nomes Parcialmente Retirados:  0  |Número de Erros:  20
Porcentagem:  9.090909090909092
Porcentagem Parcial:  0.0
cha-59974.txt
Não achou!
cha-41604.txt
Não achou!
cha-21941.txt
Não achou!
cha-56854.txt
Não achou!
cha-55913.txt
Não achou!
cha-95486.txt
Não achou!
cha-41984.txt
Não achou!
cha-46308.txt
Não achou!
cha-21081.txt
Não achou!
cha-39277.txt
Não achou!
ric-11625.txt
Nomes no HAREM:  8  | Nomes R

Nomes no HAREM:  32  | Nomes Retirados:  11  | Nomes Parcialmente Retirados:  0  |Número de Erros:  21
Porcentagem:  34.375
Porcentagem Parcial:  0.0
cha-63957.txt
Não achou!
hub-44595.txt
Nomes no HAREM:  42  | Nomes Retirados:  5  | Nomes Parcialmente Retirados:  2  |Número de Erros:  35
Porcentagem:  11.904761904761903
Porcentagem Parcial:  4.761904761904762
cha-58907.txt
Não achou!
cha-7398.txt
Não achou!
cha-71161.txt
Não achou!
cha-38280.txt
Não achou!
cha-68044.txt
Não achou!
cha-1942.txt
Não achou!
cha-90799.txt
Não achou!
cha-99489.txt
Não achou!
cha-80120.txt
Não achou!
cha-6892.txt
Não achou!
cha-12428.txt
Não achou!
cha-76608.txt
Não achou!
cha-89954.txt
Não achou!
hub-822.txt
Nomes no HAREM:  0  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:  0  |Número de Erros:  0
Porcentagem:  None
Porcentagem Parcial:  None
cha-33230.txt
Não achou!
cha-44163.txt
Não achou!
cha-90956.txt
Não achou!
cha-37598.txt
Não achou!
cha-98665.txt
Não achou!
cha-78604.txt
Não achou!
cha-74

Nomes no HAREM:  2  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:  0  |Número de Erros:  2
Porcentagem:  0.0
Porcentagem Parcial:  0.0
cha-58460.txt
Não achou!
cha-49129.txt
Não achou!
cha-55247.txt
Não achou!
cha-43265.txt
Não achou!
hub-37819.txt
Nomes no HAREM:  22  | Nomes Retirados:  3  | Nomes Parcialmente Retirados:  5  |Número de Erros:  14
Porcentagem:  13.636363636363635
Porcentagem Parcial:  22.727272727272727
cha-50571.txt
Não achou!
cha-71241.txt
Não achou!
cha-44758.txt
Não achou!
cha-46572.txt
Não achou!
cha-70643.txt
Não achou!
dav-844651.txt
Nomes no HAREM:  154  | Nomes Retirados:  31  | Nomes Parcialmente Retirados:  2  |Número de Erros:  121
Porcentagem:  20.12987012987013
Porcentagem Parcial:  1.2987012987012987
cha-52099.txt
Não achou!
cha-85541.txt
Não achou!
cha-93980.txt
Não achou!
cha-9489.txt
Não achou!
hub-77558.txt
Nomes no HAREM:  33  | Nomes Retirados:  8  | Nomes Parcialmente Retirados:  0  |Número de Erros:  25
Porcentagem:  24.242424242424242


Nomes no HAREM:  101  | Nomes Retirados:  26  | Nomes Parcialmente Retirados:  3  |Número de Erros:  72
Porcentagem:  25.742574257425744
Porcentagem Parcial:  2.9702970297029703
cha-13464.txt
Não achou!
cha-79584.txt
Não achou!
cha-85310.txt
Não achou!
cha-62689.txt
Não achou!
cha-55398.txt
Não achou!
cha-13107.txt
Não achou!
cha-33000.txt
Não achou!
cha-4874.txt
Não achou!
cha-70910.txt
Não achou!
cha-59169.txt
Não achou!
cha-27309.txt
Não achou!
cha-70520.txt
Não achou!
cha-77703.txt
Não achou!
aa56088.txt
Nomes no HAREM:  92  | Nomes Retirados:  9  | Nomes Parcialmente Retirados:  4  |Número de Erros:  79
Porcentagem:  9.782608695652174
Porcentagem Parcial:  4.3478260869565215
cha-83299.txt
Não achou!
wpt-101789548982731354.txt
Nomes no HAREM:  0  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:  0  |Número de Erros:  0
Porcentagem:  None
Porcentagem Parcial:  None
cha-60395.txt
Não achou!
hub-64260.txt
Nomes no HAREM:  0  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:

Nomes no HAREM:  13  | Nomes Retirados:  1  | Nomes Parcialmente Retirados:  0  |Número de Erros:  12
Porcentagem:  7.6923076923076925
Porcentagem Parcial:  0.0
cha-40350a.txt
Não achou!
cha-74443.txt
Não achou!
hub-28306.txt
Nomes no HAREM:  0  | Nomes Retirados:  0  | Nomes Parcialmente Retirados:  0  |Número de Erros:  0
Porcentagem:  None
Porcentagem Parcial:  None
cha-69726.txt
Não achou!
hub-71248.txt
Nomes no HAREM:  2  | Nomes Retirados:  2  | Nomes Parcialmente Retirados:  0  |Número de Erros:  0
Porcentagem:  100.0
Porcentagem Parcial:  0.0
cha-36717.txt
Não achou!
aa25258.txt
Nomes no HAREM:  48  | Nomes Retirados:  8  | Nomes Parcialmente Retirados:  0  |Número de Erros:  40
Porcentagem:  16.666666666666664
Porcentagem Parcial:  0.0
cha-95243.txt
Não achou!
cha-17365.txt
Não achou!
cha-22306.txt
Não achou!
gtpppp.txt
Nomes no HAREM:  4  | Nomes Retirados:  1  | Nomes Parcialmente Retirados:  0  |Número de Erros:  3
Porcentagem:  25.0
Porcentagem Parcial:  0.0
H2-Ren_ex6.txt

In [229]:
 sum(ar_total) , sum(ar_removidos)

(6672, 1248)

In [ ]:
# Antonio-Pedro tá pegando pedro e Antonio não. Tem que tirar o - e espaçar onde era o -.

In [ ]:
#evo = pd.read('nerp-crf.tar.gz')

#evo.to_csv('nerp-crf.tar.gz', compression='gzip')

In [ ]:
#print(conteudo[0])
#for c in conteudo:
#    print(remove_names(c))

In [ ]:
for c in conteudo:
    print(c)
    print('\n')

### Porcentagem de Acerto

In [ ]:
# Achar onde noinfo tem '<prop>' do lado

# Apos os acréscimos de noInfo correto, incorreto e dos não identificados, pegar a soma de todos e fazer regra de 3
# ou dividir o correto/total, incorreto/total e o naoID/total e contar como porcentagem, para cada texto e evolucao.

#porcentagem = 100*(part, total);
#    return 100*flutuador(part)/float(total)

In [ ]:
a = os.getcwd()+'/HAREM_EVO/HAREMNAOANOTADO.xml'
tree = ET.parse(a)
root = tree.getroot()
for doc in root:
    filename = doc.get('DOCID')
    f = open(os.getcwd()+'/HAREM_EVO/'+filename+".txt", 'w')
    for paragraph in doc:
        f.write(paragraph.text.strip('\n'))
        f.write('\n')
    f.close()